# import libraries

In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.preprocessing import image
from keras.models import load_model
from keras.models import save_model

# process class for dir

In [4]:
def load_data(data_dir):
    images = []
    labels = []

    for filename in os.listdir(os.path.join(data_dir, "images")):
        if filename.endswith('.jpg'):
            img_path = os.path.join(data_dir, "images", filename)
            label_path = os.path.join(data_dir, "labels", filename.replace('.jpg', '.txt'))

            print(f"Processing {filename}...")  # Print filename for debugging

            if os.path.isfile(label_path):
                with open(label_path, 'r') as label_file:
                    line = label_file.readline().strip()

                    if not line:
                        print(f"Empty line in file: {label_path}")
                        continue

                    try:
                        label = int(line.split()[0])
                        labels.append(label)
                    except (ValueError, IndexError) as e:
                        print(f"Error processing line in file {label_path}: {line}")
                        print(f"Error details: {e}")

                # Resize the image to the expected input shape (224, 224)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (224, 224))
                images.append(img)

    return np.array(images), np.array(labels)


# Specify your dataset directory
dataset_dir = 'TumorDetectionYolov8/OD8/Brain Tumor Detection/train'

# Load the data
images, labels = load_data(dataset_dir)

Processing volume_46_slice_61_jpg.rf.391e59308aa61a1c6184a0f2d41b62f2.jpg...
Processing volume_198_slice_79_jpg.rf.8c7c23a1ab721c83b4025175fcb7a034.jpg...
Processing volume_323_slice_31_jpg.rf.e21e68e7259f11cb7b9c1f5ceabcf503.jpg...
Processing volume_151_slice_60_jpg.rf.972f02fcc6ef2e356bdfe78ac13a35c5.jpg...
Processing volume_320_slice_105_jpg.rf.ad0d93f902dba9aa847e8928ed820611.jpg...
Processing volume_349_slice_118_jpg.rf.a66ed83ef0aaf9008ff2da2f9f242fec.jpg...
Processing volume_289_slice_104_jpg.rf.af5d214c87d9474da718cee4b61f5e44.jpg...
Processing volume_163_slice_92_jpg.rf.d530325df11d42286e19f10b046de9d5.jpg...
Processing volume_46_slice_33_jpg.rf.e2caeae2307a8685abe65d945841dc01.jpg...
Processing volume_16_slice_96_jpg.rf.7f75348a6bd47aed875448b43b09d09a.jpg...
Processing volume_28_slice_59_jpg.rf.9199e2140e4ee2dee67c35f0f33f4aef.jpg...
Processing volume_248_slice_64_jpg.rf.caaa9f6c885ea76daac79de6a4dfe131.jpg...
Processing volume_323_slice_37_jpg.rf.979704f7ff8f8081797a94870f4

# Split the dataset into training and testing sets

In [5]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# sorted labels and categories

In [6]:
# Ensure labels start from 0
min_label = np.min(y_train)
y_train = y_train - min_label
y_test = y_test - min_label

# Count the number of unique labels
num_classes = len(np.unique(y_train))

# Convert labels to categorical format
y_train_categorical = to_categorical(y_train, num_classes=num_classes)
y_test_categorical = to_categorical(y_test, num_classes=num_classes)

# build model

In [7]:
# Load the pre-trained ResNet50 model without the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers for classification
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_categorical, epochs=10, batch_size=32, validation_data=(X_test, y_test_categorical))

# Evaluate the model on the test set
accuracy = model.evaluate(X_test, y_test_categorical)[1]
print(f'Test Accuracy: {accuracy}')

Epoch 1/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 566s 3s/step - accuracy: 0.5959 - loss: 8.8022 - val_accuracy: 0.6477 - val_loss: 2291.2014
Epoch 2/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 607s 4s/step - accuracy: 0.6678 - loss: 0.6562 - val_accuracy: 0.6951 - val_loss: 0.7756
Epoch 3/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 572s 3s/step - accuracy: 0.6987 - loss: 0.5892 - val_accuracy: 0.7075 - val_loss: 0.5767
Epoch 4/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 522s 3s/step - accuracy: 0.7285 - loss: 0.5448 - val_accuracy: 0.7520 - val_loss: 0.5382
Epoch 5/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 555s 3s/step - accuracy: 0.7895 - loss: 0.4710 - val_accuracy: 0.7053 - val_loss: 0.5642
Epoch 6/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 964s 6s/step - accuracy: 0.8374 - loss: 0.3912 - val_accuracy: 0.7688 - val_loss: 0.5395
Epoch 7/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 486s 3s/step - accuracy: 0.8694 - loss: 0.3147 - val_accuracy: 0.7753 - val_loss: 0.6259
Epoch 8/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 487s 3s/step - accuracy: 0.8824 - loss: 0.3176 - val_a

In [9]:
# Preprocess the data (e.g., normalization)
X_train = X_train / 255.0
X_test = X_test / 255.0

# Ensure labels start from 0
min_label = np.min(y_train)
y_train = y_train - min_label
y_test = y_test - min_label

# Count the number of unique labels
num_classes = len(np.unique(y_train))

# Convert labels to categorical format
y_train_categorical = to_categorical(y_train, num_classes=num_classes)
y_test_categorical = to_categorical(y_test, num_classes=num_classes)

# Split the data into training and validation sets
X_train, X_val, y_train_encoded, y_val_encoded = train_test_split(X_train, y_train_categorical, test_size=0.2, random_state=42)

# Define the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_encoded, epochs=10, batch_size=32, validation_data=(X_val, y_val_encoded))

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test_categorical)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

/Users/zaina/Library/Python/3.9/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
137/137 ━━━━━━━━━━━━━━━━━━━━ 49s 353ms/step - accuracy: 0.5979 - loss: 0.7256 - val_accuracy: 0.6679 - val_loss: 0.6435
Epoch 2/10
137/137 ━━━━━━━━━━━━━━━━━━━━ 44s 321ms/step - accuracy: 0.6768 - loss: 0.5952 - val_accuracy: 0.6843 - val_loss: 0.6021
Epoch 3/10
137/137 ━━━━━━━━━━━━━━━━━━━━ 45s 328ms/step - accuracy: 0.7774 - loss: 0.4794 - val_accuracy: 0.7454 - val_loss: 0.5333
Epoch 4/10
137/137 ━━━━━━━━━━━━━━━━━━━━ 44s 319ms/step - accuracy: 0.8614 - loss: 0.3204 - val_accuracy: 0.7235 - val_loss: 0.6057
Epoch 5/10
137/137 ━━━━━━━━━━━━━━━━━━━━ 45s 325ms/step - accuracy: 0.9366 - loss: 0.1720 - val_accuracy: 0.7801 - val_loss: 0.5921
Epoch 6/10
137/137 ━━━━━━━━━━━━━━━━━━━━ 44s 321ms/step - accuracy: 0.9730 - loss: 0.0829 - val_accuracy: 0.7865 - val_loss: 0.8121
Epoch 7/10
137/137 ━━━━━━━━━━━━━━━━━━━━ 43s 311ms/step - accuracy: 0.9909 - loss: 0.0341 - val_accuracy: 0.7847 - val_loss: 0.9368
Epoch 8/10
137/137 ━━━━━━━━━━━━━━━━━━━━ 43s 316ms/step - accuracy: 0.9947 - loss: 0

In [10]:
#save keras model
model.save('local-tests/model.h5')

# Flask

In [8]:
'''app = Flask(__name__)

# Load your trained model
model = load_model('/kaggle/working/path_to_save_model.h5')

# Define a route for predictions
@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Get the image file from the request
        file = request.files['/kaggle/input/medical-image-dataset-brain-tumor-detection/Brain Tumor Detection/test/images']
        
        # Load and preprocess the image for prediction
        img = image.load_img(file, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0  # Normalize pixel values to [0, 1]

        # Make predictions
        predictions = model.predict(img_array)
        predicted_class = np.argmax(predictions)

        # Return the predicted class as JSON response
        response = {
            'class': int(predicted_class),
            'confidence': float(predictions[0][predicted_class])
        }
        return jsonify(response)

    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(port=5000, debug=False)'''

"app = Flask(__name__)\n\n# Load your trained model\nmodel = load_model('/kaggle/working/path_to_save_model.h5')\n\n# Define a route for predictions\n@app.route('/predict', methods=['POST'])\ndef predict():\n    try:\n        # Get the image file from the request\n        file = request.files['/kaggle/input/medical-image-dataset-brain-tumor-detection/Brain Tumor Detection/test/images']\n        \n        # Load and preprocess the image for prediction\n        img = image.load_img(file, target_size=(224, 224))\n        img_array = image.img_to_array(img)\n        img_array = np.expand_dims(img_array, axis=0)\n        img_array /= 255.0  # Normalize pixel values to [0, 1]\n\n        # Make predictions\n        predictions = model.predict(img_array)\n        predicted_class = np.argmax(predictions)\n\n        # Return the predicted class as JSON response\n        response = {\n            'class': int(predicted_class),\n            'confidence': float(predictions[0][predicted_class])\n   